# 📘 Data Governance with SQL

This notebook demonstrates how to use SQL to build a simple employee management database. It emphasizes data governance principles such as referential integrity, data validation, and clean schema design.

We use Python to communicate with the SQLite database and to print results in a readable format. However, all data creation, retrieval, and manipulation are done through SQL. This setup allows us to focus on learning SQL while using Python as a convenient interface.

In [ ]:
# We use sqlite3 to interact with the database 
# and pandas to format query results as tables
import sqlite3
import pandas as pd

## 🧱 1. Initialize a SQLite Database

SQLite is a lightweight and self-contained relational database engine. Unlike traditional database systems that require a separate server process, SQLite stores all data in a single file on disk and runs directly within the application that accesses it. Because of its simplicity and zero-configuration setup, SQLite is widely used in embedded systems, mobile apps, and teaching environments.

In this section, we create an in-memory SQLite database and set up a cursor object to execute SQL commands. This temporary database exists only while the notebook is running, making it ideal for experimentation and instruction.

In [ ]:
# Connect to an in-memory SQLite database
con = sqlite3.connect(':memory:')

# Create a cursor object to execute SQL commands
cur = con.cursor()

## 🛠️ 2. Define SQL Execution Helper Functions

This section defines reusable helper functions that let us execute SQL scripts and queries from files or strings. You don't need to modify or deeply understand this code. These are just functions to run SQL queries or scripts throughout the notebook.

In [ ]:
# Define a helper function to execute a SQL script inside a file
def execute_sql_script_from_file(filepath):
    try:
        with open(filepath, 'r') as f:
            sql_script = f.read()

        print(f'🖥 Executing SQL script from file {filepath}')
        cur.executescript(sql_script)
        print(f"✅ Successfully executed SQL script from file {filepath}")
    except Exception as e:
        print(f"❌ An error occurred: {e}")


# Define a helper function to execute a SQL query string
def execute_sql_query(query):
    try:
        # Execute the query
        cur.execute(query)

        if cur.description is not None:
            # Fetch all rows and column names
            rows = cur.fetchall()
            column_names = [description[0] for description in cur.description]
    
            if rows:
                print(f"✅ Query executed successfully. {len(rows)} row{'s' if len(rows) >= 2 else ''} to display.")
            else:
                print("✅ Query executed successfully. No results to display.")
    
            df = pd.DataFrame(rows, columns=column_names)
            df_sql_output_style = df.style.format(na_rep='NULL').hide(axis='index')
            display(df_sql_output_style)
        else:
            # For INSERT, UPDATE, DELETE, etc.
            print("✅ Query executed successfully. No results to display.")
    except sqlite3.Error as e:
        print(f"❌ An error occurred: {e}")

# Define a helper function to execute a SQL query inside a file
def execute_sql_query_from_file(filepath):
        with open(filepath, 'r') as f:
            query_string = f.read()

            print(f'🖥 Executing SQL query from file {filepath}')
            execute_sql_query(query_string)


## 🗂️ 3. Create and Populate Database Tables

### 3.1 Create tables using SQL scripts

The SQL statements that define our database schema are stored in a **separate `.sql` script file**. This file contains the full `CREATE TABLE` commands for `departments`, `job_titles`, and `employees`, including all primary keys, foreign keys, and check constraints. In this notebook, we simply execute that script using a helper function. If you'd like to view, edit, or study the SQL in detail, you can open the file named `sql-scripts/setup_create_tables.sql` directly in your Jupyter notebook environment or a code editor.

In [ ]:
execute_sql_script_from_file('./sql-scripts/setup_create_tables.sql')

### 3.2 Populate tables with sample rows using SQL scripts

In [ ]:
execute_sql_script_from_file('./sql-scripts/setup_populate_tables.sql')

### 3.3 Print (query) all tables within the database

In [ ]:
execute_sql_query('SELECT type, tbl_name FROM sqlite_master WHERE type="table";')

### 3.4 Query all departments

In [ ]:
execute_sql_query('SELECT * FROM departments;')

### 3.4 Query all job titles

In [ ]:
execute_sql_query('SELECT * FROM job_titles;')

### 3.4 Query first 10 rows in the employees table

In [ ]:
execute_sql_query('SELECT * FROM employees LIMIT 10;')

## 🔍 4. Using SQL to Retrieve Information with Data Governance in Mind

### 4.1 Report on-boarding employees and mask sensitive data 

- Open the SQL file at `sql-scripts/4_1_all_onboarding_employees.sql` and write a query that meets the requirements below.
- List all employees who are currently in the "On-boarding" status.
- Display each employee's full name, start date, department name, job title, pay type, and status.
- Instead of showing the actual pay rate, display the word "HIDDEN" in the pay_rate column.
- Use joins to combine data from the `employees`, `departments`, and `job_titles` tables.
- Sorting is not required.
- **Data Governance Note:** The `pay_rate` column contains sensitive compensation data. In accordance with data governance policies, sensitive information like salary should be masked or excluded from general reporting queries unless access is explicitly authorized.

In [ ]:
execute_sql_query_from_file('./sql-scripts/4_1_all_onboarding_employees.sql')

### 4.2. Identify records that should be disposed or archived

- Open the SQL file at `sql-scripts/4_2_identify_employee_rows_for_disposal.sql` and write a query that meets the requirements below.
- Assume that the company's data retention policy requires that employee records terminated before 2023 should not remain in the active database.
- List all employees who have been terminated on or before December 31, 2022.
- Display the employee's full name, department name, end date, and status.
- Use a join to include department information from the `departments` table.
- Sorting is not required.
- **Data Governance Note:** This query helps identify records that may need to be archived or deleted in accordance with the company's data retention policy.

In [ ]:
execute_sql_query_from_file('./sql-scripts/4_2_identify_employee_rows_for_disposal.sql')

### 4.3 Check for incorrect end dates

- Open the SQL file at `sql-scripts/4_3_check_end_date_anomaly.sql` and write a query that meets the requirements below.
- Find all employees whose end date is after today's date.
- Display the employee ID, full name, start date, end date, and status.
- Only include employees with an end date that is later than the current date.
- Sorting is not required.
- **Data Governance Note:** This query helps detect potential data entry errors or anomalies, such as future termination dates. Regular checks like this support data integrity and help ensure the reliability of HR records and reporting.

In [ ]:
execute_sql_query_from_file('./sql-scripts/4_3_check_end_date_anomaly.sql')

🚨 Normally, this query should return no rows, because an end date in the future would be considered a data entry error. In this case, the query returned a row for employee Christopher Crane, whose `end_date` is recorded as **2054-05-28**. This likely indicates a data entry mistake (e.g., a typo when entering the year or incorrect date formatting).

### 4.4 Check for potentially incorrect hourly rates

- Open the SQL file at `sql-scripts/4_4_check_hourly_rate_anomaly.sql` and write a query that meets the requirements below.
- List all employees who are paid on an hourly basis and have a pay rate greater than \\$10,000.
- Display the employee ID, full name, department name, job title, status, pay type, and pay rate.
- Use joins to combine data from the `employees`, `departments`, and `job_titles` tables.
- Sorting is not required.
- **Data Governance Note:** This query can help identify potentially incorrect or outlier hourly pay rates, which may signal data entry errors or policy violations. Regular checks like this support data quality monitoring by helping ensure that compensation data remains accurate, consistent, and compliant with organizational policies.

In [ ]:
execute_sql_query_from_file('./sql-scripts/4_4_check_hourly_rate_anomaly.sql')

🚨 This query checks for employees with an hourly pay rate greater than \\$10,000, which would typically be considered an anomaly. In this case, the query returned a row for employee Eric Hodges, whose `pay_rate` is recorded as \\$84,184.60. This unusually high hourly rate suggests a data entry error, such as entering an annual salary into an hourly pay field.

## 🔒 5. Using SQL to Impose Data Integrity

💡 **Note 1:** The queries in this section are **intentionally designed to fail**. They attempt to insert or update data in ways that violate the integrity rules we've defined in our schema - such as primary key constraints, foreign key references, and check conditions. These errors are a feature, not a bug: they demonstrate how well-structured databases protect themselves from invalid or inconsistent data. Understanding why these queries fail is just as important as writing queries that succeed.

**📝 Note 2:** In this section, we will use **multiline strings** in Python by wrapping our SQL queries with triple single quotes (`'''`). Triple single quotes (`'''`) or triple double quotes (`"""`) are used in Python to define **multiline string literals**. These allow you to span text across multiple lines without needing escape characters like `\n`. This is especially useful when writing SQL queries or large blocks of text.

### 5.1 Prevent a duplicate `department_id`

Try executing an `INSERT` statement with a primary key that already exists in the `departments` table. 

In [ ]:
execute_sql_query('''
    INSERT INTO departments (department_id, department_name) 
    VALUES (1, 'Accounting'); -- ⚠️ department_id == 1 already exists in the departments table
''')

🔍 Why it fails: `department_id = 1` already exists. This is a primary key constraint violation. Each primary key value should be unique.

#### Correct Query

- When inserting a new department, you should omit the `department_id` so that the database can automatically assign the next available ID.
- This prevents conflicts with existing IDs and helps maintain data integrity.

💡 Note: Running this query multiple times will keep adding an extra row to the `departments` table with duplicate `department_name` value (`'Accounting'`). This is because we have not added a `UNIQUE` constraint to the `department_name` column.

In [ ]:
execute_sql_query('''
    INSERT INTO departments (department_name) 
    VALUES ('Accounting');
''')

Check the result.

In [ ]:
execute_sql_query('''
    SELECT * FROM departments;
''')

### 5.2 Prevent a duplicate `job_title`

Try executing an `INSERT` statement with a job title that already exists in the `job_titles` table.

In [ ]:
execute_sql_query('''
    INSERT INTO job_titles (job_title) 
    VALUES ('Software Engineer'); -- ⚠️ `Software Engineer` already exists in the 
''')

🔍 Why it fails: `job_title == 'Software Engineer'` already exists in the `job_title` table. → `UNIQUE` constraint violation on `job_title`.

### 5.3 Prevent a negative `pay_rate`

Try executing an `INSERT` statement with a negative `pay_rate`.

In [ ]:
execute_sql_query('''
    INSERT INTO employees (
        employee_id,
        full_name,
        start_date,
        end_date,
        department_id,
        job_title_id,
        status,
        pay_type,
        pay_rate
    ) VALUES (
        108,
        'Ashley Potts',
        '2023-01-01',
        NULL,
        1,
        1,
        'Active',
        'Hourly',
        -25.00 -- ⚠️ pay_rate is a negative amount
    );
''')

🔍 Why it fails: `pay_rate` < 0 violates the `CHECK` constraint (`pay_rate` > 0)

### 5.4 Prevent an `end_date` that is earlier than the `start_date`

Try executing an `INSERT` statement with an `end_date` earlier than the `start_date`.

In [ ]:
execute_sql_query('''
    INSERT INTO employees (
        employee_id,
        full_name,
        start_date,
        end_date,
        department_id,
        job_title_id,
        status,
        pay_type,
        pay_rate
    ) VALUES (
        109,
        'Jenny Park',
        '2024-01-01',
        '2022-12-01', -- ⚠️ end_date is 2022, while the start date is 2024
        1,
        1,
        'Terminated',
        'Salary',
        80000
    );
''')

🔍 Why it fails: `end_date` < `start_date` → `CHECK` constraint violation (`end_date IS NULL OR end_date >= start_date`)

### 5.5 Prevent an invalid employee `status` value

Try executing an `INSERT` statement with a `status` that is not one of the allowed values.

In [ ]:
execute_sql_query('''
    INSERT INTO employees (
        employee_id,
        full_name,
        start_date,
        end_date,
        department_id,
        job_title_id,
        status,
        pay_type,
        pay_rate
    ) VALUES (
        107,
        'Ari Moss',
        '2023-09-01',
        NULL,
        1,
        1,
        'Actiive', -- ⚠️ Invalid status (typo) - should be 'Active' instead of 'Actiive'
        'Salary',
        75000
    );
''')

🔍 Why it fails: 'Actiive' is not one of the allowed values according to the `CHECK` constraint on `status` (`IN ('Active', 'On-boarding', 'Terminated')`)

## ✅ 6. Summary and Next Steps
In this notebook, we built a simple SQLite database to demonstrate key data governance principles: unique identifiers, referential integrity, and validation rules through constraints. These foundations are essential in building systems that are reliable, auditable, and secure. In the real-world, you may extend this database with more advanced concepts such as triggers, views, and real-time data validation.